In [5]:
using ITensors
using Zygote
using Random

Let's create a random complex valued 5 site MPS to begin...

In [9]:
Random.seed!(42)
s = siteinds("S=1/2", 5)
mps = randomMPS(ComplexF64, s; linkdims=4)

MPS
[1] ((dim=2|id=503|"S=1/2,Site,n=1"), (dim=4|id=418|"Link,l=1"))
[2] ((dim=4|id=418|"Link,l=1"), (dim=2|id=397|"S=1/2,Site,n=2"), (dim=4|id=439|"Link,l=2"))
[3] ((dim=4|id=439|"Link,l=2"), (dim=2|id=711|"S=1/2,Site,n=3"), (dim=4|id=491|"Link,l=3"))
[4] ((dim=4|id=491|"Link,l=3"), (dim=2|id=268|"S=1/2,Site,n=4"), (dim=2|id=588|"Link,l=4"))
[5] ((dim=2|id=588|"Link,l=4"), (dim=2|id=627|"S=1/2,Site,n=5"))


In [12]:
function complex_feature_map(x::Float64)
    s1 = exp(1im * (3π/2) * x) * cospi(0.5 * x)
    s2 = exp(-1im * (2π/2) * x) * sinpi(0.5 * x)
    return [s1, s2]
end

complex_feature_map (generic function with 1 method)

In [13]:
function sample_to_product_state(sample::Vector, site_inds::Vector{Index{Int64}})
    n_sites = length(site_inds)
    product_state = MPS(ComplexF64, site_inds; linkdims=1)
    for j=1:n_sites
        T = ITensor(site_inds[j])
        zero_state, one_state = complex_feature_map(sample[j])
        T[1] = zero_state
        T[2] = one_state
        product_state[j] = T 
    end
    return product_state
end

sample_to_product_state (generic function with 1 method)

Generate sample and encode as a product state.

In [64]:
sample = zeros(5)

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [65]:
ps = sample_to_product_state(sample, s)

MPS
[1] ((dim=2|id=503|"S=1/2,Site,n=1"),)
[2] ((dim=2|id=397|"S=1/2,Site,n=2"),)
[3] ((dim=2|id=711|"S=1/2,Site,n=3"),)
[4] ((dim=2|id=268|"S=1/2,Site,n=4"),)
[5] ((dim=2|id=627|"S=1/2,Site,n=5"),)


## Create Bond Tensor

We will create a bond tensor between sites 1 and 2

In [66]:
BT12 = mps[1] * mps[2]

ITensor ord=3 (dim=2|id=503|"S=1/2,Site,n=1") (dim=2|id=397|"S=1/2,Site,n=2") (dim=4|id=439|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

Now let's define our loss function. Since we are dealing with a single product state and bond tensor, we can make it specific:

In [94]:
function loss(B::ITensor)
    y = 1.0 # train the bond tensor to maximise overlap
    # bond tensor on sites 1 and 2
    lid = 1
    rid = 2 
    phi_tilde = ps[lid] * ps[rid] * mps[3] * ps[3] * mps[4] * ps[4] * mps[5] * ps[5] # effective input
    yhat = B * phi_tilde
    diff_mod_sq = norm(yhat[] - y)^2
    loss = 0.5 * diff_mod_sq
    return loss
end

loss (generic function with 1 method)

Test the loss function...

In [93]:
loss(BT12)

0.4120511832985574

Now construct the proposed analytic gradient for comparison against AD derived gradient:

In [79]:
function analytic_gradient(B::ITensor)
    lid = 1
    rid = 2
    phi_tilde = ps[lid] * ps[rid] * mps[3] * ps[3] * mps[4] * ps[4] * mps[5] * ps[5]
    yhat = B * phi_tilde
    y = 1.0
    dP = yhat[] - y
    grad = 0.5 * dP * conj(phi_tilde)

    return grad
end

analytic_gradient (generic function with 1 method)

# Analytic Gradient Evaluation

In [80]:
analytic_grad = analytic_gradient(BT12)

ITensor ord=3 (dim=2|id=503|"S=1/2,Site,n=1") (dim=2|id=397|"S=1/2,Site,n=2") (dim=4|id=439|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

Inspect the values...

In [81]:
collect(Iterators.flatten(analytic_grad))

16-element Vector{ComplexF64}:
 -0.027823667541818135 - 0.09315554666007617im
                  -0.0 + 0.0im
                  -0.0 + 0.0im
                  -0.0 + 0.0im
  -0.08952810956088028 + 0.03174243040135726im
                  -0.0 + 0.0im
                  -0.0 + 0.0im
                  -0.0 + 0.0im
   0.04236007657464593 - 0.028288650564501885im
                  -0.0 + 0.0im
                  -0.0 + 0.0im
                  -0.0 + 0.0im
  0.039321688985887636 - 0.022450873496918794im
                  -0.0 + 0.0im
                  -0.0 + 0.0im
                  -0.0 + 0.0im

# Zygote Gradient Evaluation

In [82]:
zygote_grad = gradient(loss, BT12)

(ITensor ord=3
Dim 1: (dim=2|id=503|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=397|"S=1/2,Site,n=2")
Dim 3: (dim=4|id=439|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×4
[:, :, 1] =
 -0.05564733508363627 - 0.18631109332015233im  -0.0 + 0.0im
                 -0.0 + 0.0im                  -0.0 + 0.0im

[:, :, 2] =
 -0.17905621912176056 + 0.06348486080271452im  -0.0 + 0.0im
                 -0.0 + 0.0im                  -0.0 + 0.0im

[:, :, 3] =
 0.08472015314929186 - 0.05657730112900377im  -0.0 + 0.0im
                -0.0 + 0.0im                  -0.0 + 0.0im

[:, :, 4] =
 0.07864337797177527 - 0.04490174699383759im  -0.0 + 0.0im
                -0.0 + 0.0im                  -0.0 + 0.0im,)

In [91]:
@show zygote_grad

zygote_grad = (ITensor ord=3
Dim 1: (dim=2|id=503|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=397|"S=1/2,Site,n=2")
Dim 3: (dim=4|id=439|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×4
[:, :, 1] =
 -0.05564733508363627 - 0.18631109332015233im  -0.0 + 0.0im
                 -0.0 + 0.0im                  -0.0 + 0.0im

[:, :, 2] =
 -0.17905621912176056 + 0.06348486080271452im  -0.0 + 0.0im
                 -0.0 + 0.0im                  -0.0 + 0.0im

[:, :, 3] =
 0.08472015314929186 - 0.05657730112900377im  -0.0 + 0.0im
                -0.0 + 0.0im                  -0.0 + 0.0im

[:, :, 4] =
 0.07864337797177527 - 0.04490174699383759im  -0.0 + 0.0im
                -0.0 + 0.0im                  -0.0 + 0.0im,)


(ITensor ord=3
Dim 1: (dim=2|id=503|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=397|"S=1/2,Site,n=2")
Dim 3: (dim=4|id=439|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×4
[:, :, 1] =
 -0.05564733508363627 - 0.18631109332015233im  -0.0 + 0.0im
                 -0.0 + 0.0im                  -0.0 + 0.0im

[:, :, 2] =
 -0.17905621912176056 + 0.06348486080271452im  -0.0 + 0.0im
                 -0.0 + 0.0im                  -0.0 + 0.0im

[:, :, 3] =
 0.08472015314929186 - 0.05657730112900377im  -0.0 + 0.0im
                -0.0 + 0.0im                  -0.0 + 0.0im

[:, :, 4] =
 0.07864337797177527 - 0.04490174699383759im  -0.0 + 0.0im
                -0.0 + 0.0im                  -0.0 + 0.0im,)

In [83]:
@show analytic_grad

analytic_grad = ITensor ord=3
Dim 1: (dim=2|id=503|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=397|"S=1/2,Site,n=2")
Dim 3: (dim=4|id=439|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×4
[:, :, 1] =
 -0.027823667541818135 - 0.09315554666007617im  -0.0 + 0.0im
                  -0.0 + 0.0im                  -0.0 + 0.0im

[:, :, 2] =
 -0.08952810956088028 + 0.03174243040135726im  -0.0 + 0.0im
                 -0.0 + 0.0im                  -0.0 + 0.0im

[:, :, 3] =
 0.04236007657464593 - 0.028288650564501885im  -0.0 + 0.0im
                -0.0 + 0.0im                   -0.0 + 0.0im

[:, :, 4] =
 0.039321688985887636 - 0.022450873496918794im  -0.0 + 0.0im
                 -0.0 + 0.0im                   -0.0 + 0.0im


ITensor ord=3 (dim=2|id=503|"S=1/2,Site,n=1") (dim=2|id=397|"S=1/2,Site,n=2") (dim=4|id=439|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

So it seems we are out by a factor of 2. The analytic gradient is half the zygote gradient.

# Test another bond tensor

In [84]:
BT23 = mps[2] * mps[3]

ITensor ord=4 (dim=4|id=418|"Link,l=1") (dim=2|id=397|"S=1/2,Site,n=2") (dim=2|id=711|"S=1/2,Site,n=3") (dim=4|id=491|"Link,l=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [85]:
function loss23(B::ITensor)
    y = 1.0 # train the bond tensor to maximise overlap
    # bond tensor on sites 1 and 2
    lid = 2
    rid = 3 
    phi_tilde = mps[1] * ps[1] * ps[lid] * ps[rid] * mps[4] * ps[4] * mps[5] * ps[5] # effective input
    yhat = B * phi_tilde
    diff_mod_sq = norm(yhat[] - y)^2
    loss = 0.5 * diff_mod_sq
    return loss
end

loss23 (generic function with 1 method)

In [86]:
loss23(BT23)

0.4120511832985574

In [87]:
function analytic_gradient23(B::ITensor)
    lid = 2
    rid = 3
    phi_tilde = mps[1] * ps[1] * ps[lid] * ps[rid] * mps[4] * ps[4] * mps[5] * ps[5] # effective input
    yhat = B * phi_tilde
    y = 1.0
    dP = yhat[] - y
    grad = 0.5 * dP * conj(phi_tilde)

    return grad
end

analytic_gradient23 (generic function with 1 method)

In [89]:
analytic_grad23 = analytic_gradient23(BT23)
@show analytic_grad23

analytic_grad23 = ITensor ord=4
Dim 1: (dim=4|id=418|"Link,l=1")
Dim 2: (dim=2|id=397|"S=1/2,Site,n=2")
Dim 3: (dim=2|id=711|"S=1/2,Site,n=3")
Dim 4: (dim=4|id=491|"Link,l=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 4×2×2×4
[:, :, 1, 1] =
   0.01011703916110151 + 0.032159849621781776im  -0.0 + 0.0im
  0.022678563632097453 - 0.013049850951575947im  -0.0 + 0.0im
  -0.02865986361853609 + 0.013492150565468316im  -0.0 + 0.0im
 0.0013413360956822478 - 0.025082537444381374im  -0.0 + 0.0im

[:, :, 2, 1] =
 -0.0 + 0.0im  -0.0 + 0.0im
 -0.0 + 0.0im  -0.0 + 0.0im
 -0.0 + 0.0im  -0.0 + 0.0im
 -0.0 + 0.0im  -0.0 + 0.0im

[:, :, 1, 2] =
 0.030913869293510886 + 0.09641329962162927im   -0.0 + 0.0im
  0.06789129363109202 - 0.039564938278620884im  -0.0 + 0.0im
 -0.08584663909490425 + 0.040991989664557756im  -0.0 + 0.0im
 0.003614037868999127 - 0.0753483847867521im    -0.0 + 0.0im

[:, :, 2, 2] =
 -0.0 + 0.0im  -0.0 + 0.0im
 -0.0 + 0.0im  -0.0 + 0.0im
 -0.0 + 0.0im  -0.0 + 0.0im
 -0.0 + 0.0im  -

ITensor ord=4 (dim=4|id=418|"Link,l=1") (dim=2|id=397|"S=1/2,Site,n=2") (dim=2|id=711|"S=1/2,Site,n=3") (dim=4|id=491|"Link,l=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [90]:
zygote_grad23 = gradient(loss23, BT23)

(ITensor ord=4
Dim 1: (dim=4|id=418|"Link,l=1")
Dim 2: (dim=2|id=397|"S=1/2,Site,n=2")
Dim 3: (dim=2|id=711|"S=1/2,Site,n=3")
Dim 4: (dim=4|id=491|"Link,l=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 4×2×2×4
[:, :, 1, 1] =
   0.02023407832220302 + 0.06431969924356355im   -0.0 + 0.0im
  0.045357127264194906 - 0.026099701903151893im  -0.0 + 0.0im
  -0.05731972723707218 + 0.026984301130936632im  -0.0 + 0.0im
 0.0026826721913644955 - 0.05016507488876275im   -0.0 + 0.0im

[:, :, 2, 1] =
 -0.0 + 0.0im  -0.0 + 0.0im
 -0.0 + 0.0im  -0.0 + 0.0im
 -0.0 + 0.0im  -0.0 + 0.0im
 -0.0 + 0.0im  -0.0 + 0.0im

[:, :, 1, 2] =
  0.06182773858702177 + 0.19282659924325854im  -0.0 + 0.0im
  0.13578258726218403 - 0.07912987655724177im  -0.0 + 0.0im
  -0.1716932781898085 + 0.08198397932911551im  -0.0 + 0.0im
 0.007228075737998254 - 0.1506967695735042im   -0.0 + 0.0im

[:, :, 2, 2] =
 -0.0 + 0.0im  -0.0 + 0.0im
 -0.0 + 0.0im  -0.0 + 0.0im
 -0.0 + 0.0im  -0.0 + 0.0im
 -0.0 + 0.0im  -0.0 + 0.0im

[:, :, 1

Same result - analytic gradient is half the zygote gradient

In [98]:
wirtinger

UndefVarError: UndefVarError: `wirtinger` not defined